In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [2]:
url = "https://www.elections.tn.gov.in/ACwise_Gendercount_19122025.aspx"

response = requests.get(url, timeout=30)
response.raise_for_status()

html = response.text

In [3]:
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table")

if table is None:
    raise ValueError("No table found on the page")


In [4]:
rows = table.find_all("tr")

data = []

for row in rows[1:]:
    cols = row.find_all("td")

    if len(cols) >= 6:
        ac_no = cols[0].get_text(strip=True)
        ac_name = cols[1].get_text(strip=True)
        total = cols[9].get_text(strip=True)

        data.append([ac_no, ac_name, total])


In [5]:
df_sir_2025 = pd.DataFrame(
    data,
    columns=[
        "ac_no",
        "ac_name",
        "total_eligible_voters_2025"
    ]
)

df_sir_2025


,ac_no,ac_name,total_eligible_voters_2025
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034
...,...,...,...
229,230,Nagercoil,243329
230,231,Colachal,251388
231,232,Padmanabhapuram,224140
232,233,Vilavancode,214360


In [6]:
# AC number
df_sir_2025["ac_no"] = pd.to_numeric(
    df_sir_2025["ac_no"],
    errors="coerce"
)

# remove commas from voter count
df_sir_2025["total_eligible_voters_2025"] = (
    df_sir_2025["total_eligible_voters_2025"]
    .str.replace(",", "", regex=False)
)

df_sir_2025["total_eligible_voters_2025"] = pd.to_numeric(
    df_sir_2025["total_eligible_voters_2025"],
    errors="coerce"
)

df_sir_2025 = df_sir_2025.dropna(
    subset=["ac_no", "total_eligible_voters_2025"]
)

df_sir_2025["ac_no"] = df_sir_2025["ac_no"].astype(int)
df_sir_2025["total_eligible_voters_2025"] = (
    df_sir_2025["total_eligible_voters_2025"].astype(int)
)

df_sir_2025.head()


,ac_no,ac_name,total_eligible_voters_2025
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034


In [7]:
df_sir_2025.shape


(234, 3)

In [8]:
conn = sqlite3.connect("../db/tn_election.db")
print("Connected to SQLite")


Connected to SQLite


In [9]:
df_sir_2025.to_sql(
    "voters_2025",
    conn,
    if_exists="replace",
    index=False
)

print("voters_2025 table created/updated")


voters_2025 table created/updated


In [10]:
pd.read_sql("SELECT * FROM voters_2025",conn)


,ac_no,ac_name,total_eligible_voters_2025
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034
...,...,...,...
229,230,Nagercoil,243329
230,231,Colachal,251388
231,232,Padmanabhapuram,224140
232,233,Vilavancode,214360
